In [38]:
import nltk
import random
from nltk.corpus import movie_reviews
import pickle
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.tokenize import word_tokenize

from nltk.classify import ClassifierI
from statistics import mode


In [39]:
class VoteClassifer(ClassifierI):
    def __init__(self, *classifiers):
        self.classifiers = classifiers
        
    def classify(self,features):
        votes = []
        for c in self.classifiers:
            v = c.classify(features)
            votes.append(v)
            return mode(votes)
    def confidence(self, features):
        votes = []
        for c in self.classifiers:
            v = c.classify(features)
            votes.append(v)
            
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        return conf


In [40]:
train_twitter_pos = open("F:/SURAJ PARAPHERNELIA/train_twitter_pos_sentiments.txt").read()
train_twitter_neg = open("F:/SURAJ PARAPHERNELIA/train_twitter_neg_sentiments.txt").read()
train_suicide = open("F:/SURAJ PARAPHERNELIA/train_suicide_stop_words_stemmed.txt").read()
train_depressed = open("F:/SURAJ PARAPHERNELIA/train_depressed_stop_words_stemmed.txt").read()


In [41]:
documents = []

for r in train_twitter_pos.split('\n'):
    documents.append((r,"pos"))

for r in train_twitter_neg.split('\n'):
    documents.append((r,"pos"))    




In [42]:
print(len(documents))

4002


In [43]:
for r in train_suicide.split('\n'):
    documents.append((r,"neg"))
    
for r in train_depressed.split('\n'):
    documents.append((r,"neg"))

In [44]:
print(len(documents))

7934


In [45]:
all_words = []

pos1 = word_tokenize(train_twitter_pos)
pos2 = word_tokenize(train_twitter_neg)

neg1 = word_tokenize(train_suicide)
neg2 = word_tokenize(train_depressed)

for w in pos1:
    all_words.append(w.lower())
    
for w in pos2:
    all_words.append(w.lower())    

for w in neg1:
    all_words.append(w.lower())

for w in pos2:
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)


In [46]:
print(len(all_words))
c = 0
for w in all_words:
    print(all_words[w],w)
   
    c+=1
    if c == 10:
        break


11562
17 o
155 alreadi
10 omg
1 juuuuuuuuuuuuuuuuussssst
5 chillin
298 today
56 hand
264 miss
3 uniform
1 hmmmm


In [47]:
word_features = list(all_words.keys())[:4000]

#print( word_features[1])

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features    

#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))


featuresets = [(find_features(rev),category) for (rev,category) in documents]

random.shuffle(featuresets)

train_size = round(0.8*len(featuresets))


training_set = featuresets[:train_size]
testing_set = featuresets[train_size:]

In [48]:
print(len(featuresets))
print(len(training_set))
print(len(testing_set))

7934
6347
1587


In [49]:
# classifier_f = open("naivebayes.pickle","rb")
# classifier = pickle.load(classifier_f)
# classifier_f.close()

classifier = nltk.NaiveBayesClassifier.train(training_set)

print("original naive bayes accuracy",nltk.classify.accuracy(classifier,testing_set)*100)

classifier.show_most_informative_features(15)

# =============================================================================
# #save_classifier = open("naivebayes.pickle","wb")
# #pickle.dump(classifier,save_classifier)             THIS IS HOW WE DUMP A PICKLE
# #save_classifier.close()
# 
# =============================================================================



#                               MultinomialNB,  GaussianNB, BernoulliNB







#GaussianNB_classifer = SklearnClassifier(GaussianNB())
#GaussianNB_classifer.train(training_set)
#print("MNB_classifer_accuracy percent:",(nltk.classify.accuracy(GaussianNB_classifer,testing_set)*100))


original naive bayes accuracy 58.60113421550095
Most Informative Features
                       j = True              neg : pos    =      2.7 : 1.0
                       x = True              neg : pos    =      2.6 : 1.0
                       v = True              neg : pos    =      2.0 : 1.0
                       ' = True              neg : pos    =      1.8 : 1.0
                       q = True              neg : pos    =      1.7 : 1.0
                       p = False             pos : neg    =      1.6 : 1.0
                       v = False             pos : neg    =      1.6 : 1.0
                       f = True              neg : pos    =      1.5 : 1.0
                       k = False             pos : neg    =      1.5 : 1.0
                       f = False             pos : neg    =      1.5 : 1.0
                       p = True              neg : pos    =      1.4 : 1.0
                       k = True              neg : pos    =      1.4 : 1.0
                       y =

In [50]:
SGDClassifier_classifer = SklearnClassifier(SGDClassifier())
SGDClassifier_classifer.train(training_set)
print("SGDClassifier_accuracy percent:",(nltk.classify.accuracy(SGDClassifier_classifer,testing_set)*100))


F:\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_accuracy percent: 54.694391934467546


In [51]:
MNB_classifer = SklearnClassifier(MultinomialNB())
MNB_classifer.train(training_set)

print("MNB_classifer_accuracy percent:",(nltk.classify.accuracy(MNB_classifer,testing_set)*100))


MNB_classifer_accuracy percent: 60.995589161940764


In [52]:
BernoulliNB_classifer = SklearnClassifier(BernoulliNB())
BernoulliNB_classifer.train(training_set)
print("BernoulliNB_accuracy percent:",(nltk.classify.accuracy(BernoulliNB_classifer,testing_set)*100))


BernoulliNB_accuracy percent: 58.79017013232514


In [53]:
LogisticRegression_classifer = SklearnClassifier(LogisticRegression())
LogisticRegression_classifer.train(training_set)
print("LogisticRegression_accuracy percent:",(nltk.classify.accuracy(LogisticRegression_classifer,testing_set)*100))

LogisticRegression_accuracy percent: 69.12413358538122


In [54]:
SVC_classifer = SklearnClassifier(SVC())
SVC_classifer.train(training_set)
print("SVC_classifer_accuracy percent:",(nltk.classify.accuracy(SVC_classifer,testing_set)*100))


SVC_classifer_accuracy percent: 64.71329552614998


In [55]:
LinearSVC_classifer = SklearnClassifier(LinearSVC())
LinearSVC_classifer.train(training_set)
print("LinearSVC_accuracy percent:",(nltk.classify.accuracy(LinearSVC_classifer,testing_set)*100))


LinearSVC_accuracy percent: 69.25015752993069


In [56]:
NuSVC_classifer = SklearnClassifier(NuSVC())
NuSVC_classifer.train(training_set)
print("NuSVC_accuracy percent:",(nltk.classify.accuracy(NuSVC_classifer,testing_set)*100))


NuSVC_accuracy percent: 39.38248267170763


In [57]:
voted_classifier = VoteClassifer(classifier,MNB_classifer,BernoulliNB_classifer,SGDClassifier_classifer,LinearSVC_classifer,NuSVC_classifer,SVC_classifer)

print("voted classifier_accuracy percent:",(nltk.classify.accuracy(voted_classifier,testing_set)*100))

print("classification:",voted_classifier.classify(testing_set[0][0]),"confidence %:",voted_classifier.confidence(testing_set[0][0]))
print("classification:",voted_classifier.classify(testing_set[1][0]),"confidence %:",voted_classifier.confidence(testing_set[1][0]))
print("classification:",voted_classifier.classify(testing_set[2][0]),"confidence %:",voted_classifier.confidence(testing_set[2][0]))


voted classifier_accuracy percent: 58.60113421550095
classification: neg confidence %: 0.7142857142857143
classification: pos confidence %: 0.5714285714285714
classification: pos confidence %: 1.0
